In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

In [2]:
class CharacterTable:
    """与えられた文字のセット:
    + One Hot 表現にエンコードする
    + ワンホットまたは整数表現を文字出力にデコードする
    + 確率ベクトルを文字出力にデコードします
    """

    def __init__(self, chars):
        """文字テーブルを初期化
        # 引数
            chars: 入力に表示できる文字。
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """与えられた文字列 C の one-hot エンコード
        # 引数
            C: 文字列。エンコードされる。
            num_rows: 返されたワンホットエンコーディングの行数。これは、各データの行数を同じに保つために使用される。
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """指定されたベクトルまたは二次元配列を文字出力にデコードします。
        # 引数
            x: 確率またはワンホット表現のベクトルまたは二次元配列;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))

Generating data...
Total questions: 50000


In [3]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)

Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [4]:
print("Build model...")
num_layers = 1  # Try to add more LSTM layers!

model = keras.Sequential()
# LSTMを使用して入力シーケンスを「エンコード」する
# 入力シーケンスの長さが可変である状況では、input_shape =(None、num_feature)
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# デコーダーのRNNの入力として、タイムステップごとにRNNの最後の出力を繰り返し提供します。
# 'DIGITS + 1'を繰り返します。これは出力の最大長です。たとえば、DIGITS = 3の場合、最大出力は999 + 999 = 1998です。
model.add(layers.RepeatVector(DIGITS + 1))
# デコーダーRNNは、スタックされた複数のレイヤーまたは単一のレイヤーにすることができます。
for _ in range(num_layers):
    # return_sequencesをTrueに設定すると、最後の出力だけでなく、これまでのすべての出力が（num_samples、timesteps、output_dim）の形式で返されます。
    # 以下のTimeDistributedは、最初のディメンションがタイムステップであると想定しているため、これが必要です。
    model.add(layers.LSTM(128, return_sequences=True))

# 入力のすべての時間スライスに密なレイヤーを適用します。出力シーケンスの各ステップで、どの文字を選択するかを決定します。
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               72192     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense (Dense)                (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [5]:
epochs = 30
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 1
1407/1407 [==============================] - 29s 19ms/step - loss: 1.8824 - accuracy: 0.3248 - val_loss: 1.5426 - val_accuracy: 0.4265
Q 63+825  T 888  ☒ 901 
Q 474+28  T 502  ☒ 496 
Q 70+22   T 92   ☒ 10  
Q 54+978  T 1032 ☒ 101 
Q 8+706   T 714  ☒ 886 
Q 262+6   T 268  ☒ 666 
Q 51+548  T 599  ☒ 556 
Q 676+597 T 1273 ☒ 1266
Q 311+26  T 337  ☒ 223 
Q 36+95   T 131  ☒ 166 

Iteration 2
1407/1407 [==============================] - 24s 17ms/step - loss: 1.3336 - accuracy: 0.5017 - val_loss: 1.1797 - val_accuracy: 0.5517
Q 88+804  T 892  ☒ 874 
Q 680+117 T 797  ☒ 744 
Q 16+808  T 824  ☒ 829 
Q 19+48   T 67   ☒ 59  
Q 698+205 T 903  ☒ 774 
Q 15+305  T 320  ☒ 349 
Q 93+68   T 161  ☒ 144 
Q 72+717  T 789  ☒ 777 
Q 549+659 T 1208 ☒ 1174
Q 35+323  T 358  ☒ 349 

Iteration 3
1407/1407 [==============================] - 24s 17ms/step - loss: 1.0517 - accuracy: 0.6088 - val_loss: 0.9683 - val_accuracy: 0.6392
Q 252+34  T 286  ☒ 283 
Q 77+669  T 746  ☒ 735 
Q 69+528  T 597  ☒ 695 
Q 8+